### Created by Ryan Murphy and Seandatasci

In [1]:
import plotly_express as px
import pandas as pd
import numpy as np
import ipywidgets as ipyw
import plotly.offline as py
import plotly.graph_objs as go
import re

In [2]:
#set the token for the mapbox
px.set_mapbox_access_token(open("mapbox_token.txt").read())

In [3]:
#read in the dataset
blm = pd.read_csv('police_killings_cleaned.csv', encoding='latin-1')

In [7]:
original_shooting_map = px.scatter_mapbox(blm, lat="latitude", lon="longitude", color = 'raceethnicity', 
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15
                                 ,hover_name = "raceethnicity", height = 600, width = 1100, zoom=2)
original_shooting_map

In [4]:
class design_hoverbox(object):
    
    
    def __init__(self, data, graph_object, vars_wanted = None, labs = None):
        """also need to set a default for
        the vars wanted...what if it is passed in as none!"""
        
        #if user passes in list of custom labels, ensure 
        #length of lables == length of variables specified by user. 
        if labs != None:
            len(vars_wanted) == len(labs)
        else:
            self.labs = labs
        
        #store a deep copy of the dataframe passed in:
        self.df = data.copy(deep=True)
        
        #add a reference index to the dataframe:
        self.df['ref_index'] = data.index
        
        #store a deep copy of the plotly graph object passed in
        self.graph = graph_object
        
        #store variables selected for inclusion in the hover box
        self.vars = vars_wanted
        
        #instantiate cat_name. If the user specified a group to color
        #or fill graph attributes by, this is set to cat_name
        self.cat_name = None
        
        self.new_hover_box()
        
        
    def __get_hover_bool__(self, group_i):
        """Adds boolean to hover_bool, indicating whether or not each 
        group has an associated hover text."""
        
        try:
            return any(self.graph.data[group_i].hovertext != None)
        
        except:
            return False

            
    def __get_group_inds__(self, group_i):
        """Sets the customdata for each group in the graph object to be
        the indexes that correspond to each row in that group"""
            
        #group_i_val is the name of the variable corresponding to the group_i
        group_i_val = re.split("=", self.graph.data[group_i].name)[1]

        #inds stores the indexes of the rows that correspond to each row in the ith group
        group_inds = self.df[self.df[self.cat_name] == group_i_val].ref_index

        #store a list of indexes corresponding to the rows that
        #belong to group_i in the customdata value for that group. 
        self.graph.data[group_i]["customdata"] = group_inds
        
         
    def __get_new_html__(self, point_index, group_index):
        """Returns a new htlm string for the hover box.
        uses the indexes created in get_custs to index the data frame
        and adds the information pertaining to that individual to the hover box"""

        #instantiate an empty string to hold html template
        to_ret = ''
        
        #for each data point in the group, create a new htmltemplate
        #that includes the variables specified by the user.
        for val_ind, val in enumerate(self.vars):
            
            new_value = self.df.iloc[[point_index]][val][point_index]
            
            to_ret += '<br>' + val + ': ' + str(new_value)
            
        return to_ret
    
    def __change_hovertemplate__(self):
        """Changes the hovertemplate for each class.
        The hovertemplate for each data point is hard-coded in the customdata entry 
        for each group (stored in a list).
        The new hovertemplate calls customdata to retrieve the appropriate
        html string for each point."""
        
        for group_i, data_struct_i in enumerate(self.graph.data):
            
            if self.__get_hover_bool__(group_i):
                data_struct_i.hovertemplate = "<b>%{hovertext}</b><br><b>%{customdata}"
                
            else: 
                data_struct_i.hovertemplate = "<b>%{customdata}"
             
    def new_hover_box(self):
        """Customizes a hover box object for a plotly graph object"""
        
        #if the graph has a grouped object (i,e., color = x),
        #call the get group inds function for each group
        if len(self.graph.data) > 1:
            self.cat_name = re.split("=", self.graph.data[1].name)[0]
            [self.__get_group_inds__(group_i) for group_i in range(len(self.graph.data))]
            
        else:
            #if the graph has no groups, set custom data to be a list 
            #of indexes for all the data points
            self.graph.data[0]["customdata"] = self.df.index
            
        #Retrive a new html string for each data point,
        #which includes the variables selected by the user. 
        for group_i, data_struct_i in enumerate(self.graph.data):
            
            #change customdata entry in each group to be a list of
            #hovertemplates corresponding to each row in that group
            data_struct_i.customdata = [self.__get_new_html__(point_index, group_i)
                                       for point_index in data_struct_i.customdata]
        
        #change the html code on the hovertemplate 
        self.__change_hovertemplate__()
        
        return self.graph

In [10]:
shooting_map = px.scatter_mapbox(blm, lat="latitude", lon="longitude", color = 'raceethnicity', 
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15
                                 ,hover_name = "raceethnicity", height = 600, width = 1100, zoom=2)

shooting_map.layout.title = go.layout.Title(text = "Map Quest",x=.35)

#adjusting the legend of the plot
shooting_map.layout.legend.font = {'color':"black",'family': 'Serif', 'size': 15}
shooting_map.layout.legend.bgcolor = 'white'#change the color of the box with this. 
# shooting_map.layout.legend.bordercolor = 'black'
# shooting_map.layout.legend.borderwidth = 2
shooting_map.layout.annotations = [dict(text = 'Ethnicity', x=1.2, y=1.05, showarrow = False,
                                        font = {'color':"black",'family': 'Serif', 'size': 20})]

design_hoverbox(data=blm, graph_object=shooting_map, vars_wanted=["gender","cause"])

In [11]:
shooting_map